# Tracer analysis in NWES for different initital displacement
In the paper we compare the difference between MRG-particles and tracer praticles to the difference between (initially) displaced tracers and tracer particles. We use an initial displacement of d=300m (5 min x 1 m/s), based on the integration timestep of 5 min and typical velocity scales of 1 m/s. This is a quite ad-hoc value so here we show how this measure changes if other initial displacements are used. 

In [ ]:
#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import colormaps
import cartopy.crs as ccrs #for plotting on map
import cartopy as cart
from datetime import datetime, timedelta

# import custom functions 
import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
from analysis_functions_xr import trajectory_length, Haversine_list, calc_tidal_av
from analysis_functions import make_PDF, Haversine
from particle_characteristics_functions import factor_drag_white1991

# plotstyle: 
plt.style.use('../python_style_Meike.mplstyle')
Viridis = colormaps['viridis']
ViridisList = Viridis(np.linspace(0.2, 1.0, 6))#Magma(np.linspace(0.2, 0.9, 5))
Magma = colormaps['magma']
Magmalist = Magma(np.linspace(0.2, 0.9, 6))

In [ ]:
# set needed constants
Rearth = 6371 * 10**3 # in m,
deg2rad = np.pi / 180.
sec_in_hours= 3600
diameter = 0.2 #m
rho_water = 1027 # kg/m3 https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
dynamic_viscosity_water = 1.41 * 10**(-3) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water



In [ ]:
# set simulation parameters to read in 
displacement_list = [10,100,300,500,1000,2000]

starttimes = np.array([datetime(2023, 9, 1, 0, 0, 0, 0), 
                       datetime(2023, 10, 1, 0, 0, 0, 0),
                       datetime(2023, 11, 1, 0, 0, 0, 0),
                       datetime(2023, 12, 1, 0, 0, 0, 0),
                       datetime(2024, 1, 1, 0, 0, 0, 0),
                       datetime(2024, 2, 1, 0, 0, 0, 0)])
runtime =  timedelta(days=30)
nparticles = 52511
chunck_time = 100
loc = 'NWES'
land_handling = 'anti_beaching'

# set filenames
base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'
basefile_tracer  = (base_directory + '{particle_type}/{loc}_'
                        'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                        'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_{land_handling}.zarr')
basefile_tracer_random = (base_directory + 'tracer_random/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                   '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_d{d:04d}_{land_handling}.zarr')
basefiles = {'tracer':basefile_tracer,
             'tracer_random':basefile_tracer_random
             }

particle_types = ['tracer','tracer_random'] # 
simtype = {'tracer':'tracer','tracer_random':'displaced tracer'}                       

In [ ]:
# read in data
data = {}
for pt in particle_types:
    data[pt]={}
    if(pt =='tracer_random'):
        for d in displacement_list:
            data[pt][d]={}
    else:
        data[pt][None]={}

for pt in particle_types:
     for starttime in starttimes:
        date = f'{starttime.year:04d}/{starttime.month:02d}'
        endtime = starttime + runtime
        if(pt =='tracer_random'):
            for d in displacement_list:
                file = basefiles[pt].format(loc=loc,
                                                y_s=starttime.year,
                                                m_s=starttime.month,
                                                d_s=starttime.day,
                                                y_e=endtime.year,
                                                m_e=endtime.month,
                                                d_e=endtime.day,
                                                d=d,
                                                land_handling = land_handling, 
                                                particle_type = pt)
                ds = xr.open_dataset(file,
                                     engine='zarr',
                                     chunks={'trajectory':nparticles, 'obs':chunck_time},
                                     drop_variables=['z'],
                                     decode_times=False) #,decode_cf=False)

                data[pt][d][date]= ds 
        else:
            file = basefiles[pt].format(loc=loc,
                                                y_s=starttime.year,
                                                m_s=starttime.month,
                                                d_s=starttime.day,
                                                y_e=endtime.year,
                                                m_e=endtime.month,
                                                d_e=endtime.day,
                                            
                                                land_handling = land_handling, 
                                                particle_type = pt)
            ds = xr.open_dataset(file,
                                 engine='zarr',
                                 chunks={'trajectory':nparticles, 'obs':chunck_time},
                                 drop_variables=['z'],
                                 decode_times=False) #,decode_cf=False)

            data[pt][None][date]= ds 

In [ ]:
year=2023
month=9
date = f'{year:04d}/{month:02d}'
legend = []
fig,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.coastlines()
idstart = 25 
ax.add_feature(cart.feature.LAND, facecolor='lightgrey')

for pt in particle_types:
    if(pt == 'tracer_random'):
        for d, color in zip(displacement_list,Magmalist):
            ax.plot(data[pt][d][date].lon[idstart,0::1].T,
                    data[pt][d][date].lat[idstart,0::1].T,
                    '--',
                    color=color);
            legend.append(simtype[pt]+f', d ={d} m')
    else:
        ax.plot(data[pt][None][date].lon[idstart,0::1].T,
                    data[pt][None][date].lat[idstart,0::1].T,
                    '-',
                    color='k',zorder=-20);
        legend.append(simtype[pt])
ax.legend(legend,fontsize=15,handlelength=1.5)
ax.set_xlim(-14.6,-11.2)
ax.set_ylim(54.4,56)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
        linewidth=0, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 18}
gl.ylabel_style =  {'size': 18}


In [ ]:
# add tidal average lons and lats
window = 25 
startnan=np.full((nparticles,25),np.nan)
endnan=np.full((nparticles,80),np.nan)
Tend=720

for starttime in starttimes:
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}'
    for pt in particle_types:
        if(pt == 'tracer_random'):
            for d in displacement_list:
                ds=data[pt][d][date]
                da_lon_tidal_av = calc_tidal_av(ds.lon,window)
                da_lat_tidal_av = calc_tidal_av(ds.lat,window)

                ds = ds.assign(lon_tidal_av = da_lon_tidal_av)
                ds = ds.assign(lat_tidal_av = da_lat_tidal_av)
                data[pt][d][date]=ds
        else:
            ds=data[pt][None][date]
            da_lon_tidal_av = calc_tidal_av(ds.lon,window)
            da_lat_tidal_av = calc_tidal_av(ds.lat,window)

            ds = ds.assign(lon_tidal_av = da_lon_tidal_av)
            ds = ds.assign(lat_tidal_av = da_lat_tidal_av)
            data[pt][None][date]=ds


In [ ]:
# along track distance calculation
for pt in particle_types:
    for strattime in starttimes:
        year = starttime.year
        month = starttime.month
        date = f'{year:04d}/{month:02d}'
        if(pt == 'tracer_random'):
            for d in displacement_list:
                ds =data[pt][d][date]
                traj_length = trajectory_length(ds.lon, ds.lat)
                ds = ds.assign(traj_length = traj_length)
                traj_length_tidal_av = trajectory_length(ds.lon_tidal_av[:,window:], ds.lat_tidal_av[:,window:])
                data[pt][d][date] = ds.assign(traj_length_tidal_av = traj_length_tidal_av)
        else:
            ds =data[pt][None][date]
            traj_length = trajectory_length(ds.lon, ds.lat)
            ds = ds.assign(traj_length = traj_length)
            traj_length_tidal_av = trajectory_length(ds.lon_tidal_av[:,window:], ds.lat_tidal_av[:,window:])
            data[pt][None][date] = ds.assign(traj_length_tidal_av = traj_length_tidal_av)
        





In [ ]:
# plot pdf difference trajectory length tracer and displaced tracers
Tmax =719
fig, ax = plt.subplots()
data_arrays_tracer =  list(data['tracer'][None].values())
combined_data_tracer = xr.concat(data_arrays_tracer, dim="starting_days")
pt = 'tracer_random'
legend=[]
for d, color in zip(displacement_list,Magmalist):
    data_arrays_random = list(data[pt][d].values())
    combined_data_random = xr.concat(data_arrays_random,dim="starting_days")
    diff = combined_data_random.traj_length - combined_data_tracer.traj_length
    average_data = diff.mean(dim='starting_days',skipna=True)
    bins, pdf = make_PDF(average_data[:,Tmax].values.flatten(),101,norm=True,vmin=-20,vmax=20)
    ax.plot(bins,pdf,'--',color=color)
    legend.append(f'displacement = {d} m')

ax.legend(legend,fontsize=18,handlelength = 1.5)
ax.set_ylabel("PDF")
ax.set_xlabel("trajectory length difference \n displaced tracer and tracer [km]")






In [ ]:
# relative dispersion
for starttime in starttimes:
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}'
    ds_tracer = data['tracer'][None][date]
    pt = 'tracer_random'
    for d in displacement_list:
        ds = data[pt][d][date]
        dist= Haversine(ds['lon'],ds['lat'],ds_tracer['lon'],ds_tracer['lat'])
        ds=ds.assign(dist = dist)
        ds=ds.rename({'dist':'dist_to_tracer'})
        #tidal av
        dist_tidal_av= Haversine(ds['lon_tidal_av'],ds['lat_tidal_av'],
                            ds_tracer['lon_tidal_av'],ds_tracer['lat_tidal_av'])
        ds=ds.assign(dist_tidal_av = dist_tidal_av)
        data[pt][d][date]=ds.rename({'dist_tidal_av':'dist_to_tracer_tidal_av'})


In [ ]:
Tmax = 720
tlist=np.arange(0,Tmax,1)/24
fig,ax=plt.subplots()
legend=[]
pt = 'tracer_random'
for d, color in zip(displacement_list,Magmalist):
    data_arrays_random=list(data[pt][d].values())
    combined_data_random = xr.concat(data_arrays_random, dim="starting_days") 
    mean=combined_data_random['dist_to_tracer'].mean(dim='trajectory',skipna=True)
    average_mean = mean.mean(dim="starting_days",skipna=True)
    average_std = mean.std(dim="starting_days",skipna=True)/np.sqrt(starttimes.size)
    ax.plot(tlist, average_mean[:Tmax],'--', color=color)
    ax.fill_between(tlist, average_mean[:Tmax]+average_std[:Tmax],  average_mean[:Tmax]-average_std[:Tmax],color=color,alpha=0.2,label='_nolegend_')#,alhpa=0.2)
    legend.append(f'displacement = {d} m')


ax.legend(legend,fontsize=18,handlelength=1.5)
ax.set_xlabel('t [days]')
ax.set_ylabel('relative distance between \n displaced tracer and tracer [km]')
# ax.set_yscale('log')
# ax.set_xscale('log')
# ax.set_title('coriolis forcing',fontsize=18)